# Feature engineering

**Purpose of script:**

Creating new features 

In [3]:
# imports
import pandas as pd
import numpy as np

In [4]:
# paths
df_path = r"../Data/combined/"

out_path = r"../Data/combined/pandas_extended"

In [5]:
# data load 
#df = pd.read_csv(df_path + 'melt_2019-07-01.csv')
df = pd.read_parquet(df_path + 'melt_2019-07-02.parquet.gzip')

## Features:

In script 05:
- Column and row numbers
- Neighboring pixels
- Water pixels from mw removed 



Date

In [6]:
df['date'] = '2019-07-02'
df["date"]= pd.to_datetime(df["date"])
# df['month'] = df["date"].dt.month

Aggregated/pooled values

In [7]:
# delet v5? same as mw value

In [8]:
cols = ['v1', 'v2', 'v3', 'v4', 'v5', 'v6', 'v7', 'v8', 'v9']

In [9]:
df[cols] = df[cols].replace(-1, np.NaN) # to skip -1 when calculating mean
df['mean'] = df[cols].mean(axis = 1) # mean value of 9 pixels around

Elevation data

In [10]:
import xarray
import rasterio

In [11]:
path_elevation =  r"../Data/elevation_data/Greenland_1km.tif"
data_elevation = xarray.open_dataarray(path_elevation)


In [12]:
with rasterio.open(path_elevation) as dataset_elev:
    print(dataset_elev.crs)
# reference system matches opt an mw data

print(data_elevation.shape) # shape seems same as optical data

EPSG:3413
(1, 2687, 1487)


In [13]:
# import matplotlib.pyplot as plt
# fig, axes = plt.subplots(ncols=1, figsize=(4,5))
# data_elevation.plot(ax=axes) # , cmap = cmap
# plt.draw()

In [14]:
df_elevation = data_elevation.to_dataframe()
df_elevation = df_elevation.reset_index()
df_elevation = df_elevation[['x', 'y', 'band_data']]
df_elevation.rename({'band_data': 'elevation_data'}, axis=1, inplace=True)

In [15]:
df_elevation

,x,y,elevation_data
0,-636500.0,-662500.0,NaN
1,-635500.0,-662500.0,NaN
2,-634500.0,-662500.0,NaN
3,-633500.0,-662500.0,NaN
4,-632500.0,-662500.0,NaN
...,...,...,...
3995564,845500.0,-3348500.0,NaN
3995565,846500.0,-3348500.0,NaN
3995566,847500.0,-3348500.0,NaN
3995567,848500.0,-3348500.0,NaN


In [16]:
# merge
df_with_elevation = pd.merge(df, df_elevation, how = 'left', on = ['y', 'x']) # left smaller mw, right - opt

In [17]:
df_with_elevation

,x,y,mw_value,opt_value,col,row,v1,v2,v3,v4,v5,v6,v7,v8,v9,date,mean,elevation_data
0,206500.0,-677500.0,0.0,0.148801,843,15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2019-07-02,0.0,150.0
1,186500.0,-678500.0,0.0,0.015469,823,16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2019-07-02,0.0,150.0
2,187500.0,-678500.0,0.0,0.013495,824,16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2019-07-02,0.0,150.0
3,188500.0,-678500.0,0.0,0.101540,825,16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2019-07-02,0.0,150.0
4,190500.0,-678500.0,0.0,0.000418,827,16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2019-07-02,0.0,150.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1026834,-11500.0,-3308500.0,0.0,0.001044,625,2646,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,2019-07-02,0.0,110.0
1026835,42500.0,-3314500.0,0.0,0.008728,679,2652,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2019-07-02,0.0,130.0
1026836,58500.0,-3314500.0,0.0,0.013714,695,2652,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2019-07-02,0.0,150.0
1026837,54500.0,-3318500.0,0.0,0.013711,691,2656,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2019-07-02,0.0,150.0


Distance from margin/shore

In [18]:
# add if coast column - if at least one na but not all 

Geohash

In [19]:
# convert x and y to useful sections of geodata

Write

In [20]:
df_with_elevation.to_parquet(out_path + 'melt_2019-07-02_extended.parquet.gzip', index= False)